# K-means

In [3]:
import pandas as pd
from numpy import array
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark import SparkContext
from pyspark import SQLContext
import py4j

In [7]:
moviepath='C:/Users/SALONI/Documents/Semester 4/Cloud Computing/Assignment 2/movies.txt'
userpath='C:/Users/SALONI/Documents/Semester 4/Cloud Computing/Assignment 2/users.txt'
ratingpath='C:/Users/SALONI/Documents/Semester 4/Cloud Computing/Assignment 2/ratings.txt'


In [8]:
moviefile= pd.read_table(moviepath,sep='::',names=['movie_id','movie_name', 'genre'],encoding='iso-8859-1')

C:\Users\SALONI\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [9]:
userfile= pd.read_table(userpath,sep='::',names=['UserID','Gender','Age','Occupation','Zip-code'],encoding='iso-8859-15')


C:\Users\SALONI\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [10]:
ratingfile= pd.read_table(ratingpath,sep='::',names=['UserID','MovieID','Rating','Timestamp'],encoding='iso-8859-15')


C:\Users\SALONI\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [11]:
ratingfile['MovieID'].nunique()

3706

In [12]:
moviefile['movie_id'].nunique()


3883

In [13]:
file_genre= pd.merge(left=ratingfile, right=moviefile, how='left', left_on='MovieID', right_on='movie_id') 


In [14]:
file_genre=file_genre.drop(['Timestamp','movie_id'],axis=1)


In [15]:
file_genre

,UserID,MovieID,Rating,movie_name,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...
1000204,6040,1091,1,Weekend at Bernie's (1989),Comedy
1000205,6040,1094,5,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,Sophie's Choice (1982),Drama


In [16]:
genre_list= set()


In [17]:
for g in file_genre['genre'].str.split('|'):
    genre_list= genre_list.union(set(g))

In [18]:
for g in genre_list:
    file_genre[g]=0


In [19]:
i=file_genre.index.min()
#type(i)
while i<=file_genre.index.max():
    for g in file_genre.loc[i,'genre'].split('|'):
        file_genre.at[i,g]=1
    i=i+1

In [20]:
file_genre= file_genre.drop(['movie_name'], axis=1)


In [21]:
backupfile=file_genre


In [22]:
file_genre=file_genre.drop(['genre'], axis=1)


In [23]:
file_genre.to_csv(r'C:/Users/SALONI/Documents/Semester 4/Cloud Computing/Assignment 2/file_genre',index = False)


In [24]:
file_genre= pd.read_csv('C:/Users/SALONI/Documents/Semester 4/Cloud Computing/Assignment 2/file_genre')


In [25]:
sqlctx= SQLContext(sc)


In [26]:
SpData= sqlctx.createDataFrame(file_genre)
SpData

DataFrame[UserID: bigint, MovieID: bigint, Rating: bigint, Sci-Fi: bigint, Adventure: bigint, Horror: bigint, Film-Noir: bigint, Action: bigint, Musical: bigint, Romance: bigint, Crime: bigint, Children's: bigint, Comedy: bigint, Mystery: bigint, Drama: bigint, Animation: bigint, Western: bigint, Documentary: bigint, Thriller: bigint, Fantasy: bigint, War: bigint]

In [27]:
ignore=['UserID','MovieID', 'Rating']


In [28]:
assembler=VectorAssembler(inputCols=[x for x in file_genre if x not in ignore], outputCol='features')
assembler


VectorAssembler_3c519263ddfd

In [29]:
tdata=assembler.transform(SpData)
tdata

DataFrame[UserID: bigint, MovieID: bigint, Rating: bigint, Sci-Fi: bigint, Adventure: bigint, Horror: bigint, Film-Noir: bigint, Action: bigint, Musical: bigint, Romance: bigint, Crime: bigint, Children's: bigint, Comedy: bigint, Mystery: bigint, Drama: bigint, Animation: bigint, Western: bigint, Documentary: bigint, Thriller: bigint, Fantasy: bigint, War: bigint, features: vector]

In [30]:
least_ssee=[]
for i in range(25,35):
    kmeans= KMeans().setK(i).setSeed(134)
    model= kmeans.fit(tdata)
    sse=model.summary.trainingCost
    least_ssee.append([sse,i])

In [31]:
least_ssee


[[523230.428654439, 25],
 [517262.8526350386, 26],
 [507205.393494464, 27],
 [503676.65993441606, 28],
 [499280.84063860576, 29],
 [483928.6699055842, 30],
 [486010.92184351385, 31],
 [469261.9184572046, 32],
 [456607.6482565256, 33],
 [414578.07381927065, 34]]

In [32]:
kmeans= KMeans().setK(29).setSeed(134)


In [33]:
model=kmeans.fit(tdata)


In [37]:
prediction=model.transform(tdata)
prediction.show()


+------+-------+------+------+---------+------+---------+------+-------+-------+-----+----------+------+-------+-----+---------+-------+-----------+--------+-------+---+--------------------+----------+
|UserID|MovieID|Rating|Sci-Fi|Adventure|Horror|Film-Noir|Action|Musical|Romance|Crime|Children's|Comedy|Mystery|Drama|Animation|Western|Documentary|Thriller|Fantasy|War|            features|prediction|
+------+-------+------+------+---------+------+---------+------+-------+-------+-----+----------+------+-------+-----+---------+-------+-----------+--------+-------+---+--------------------+----------+
|     1|   1193|     5|     0|        0|     0|        0|     0|      0|      0|    0|         0|     0|      0|    1|        0|      0|          0|       0|      0|  0|     (18,[11],[1.0])|        12|
|     1|    661|     3|     0|        0|     0|        0|     0|      1|      0|    0|         1|     0|      0|    0|        1|      0|          0|       0|      0|  0|(18,[5,8,12],[1.0...|  

In [38]:
drop_col=['Documentary','Mystery','Romance','Crime','Western','Horror','Film-Noir','Adventure','Animation','Sci-Fi','War','Thriller','Drama',"Children's",'Comedy','Musical','Fantasy','Action']

In [39]:
prediction= prediction.drop(*drop_col)
pred_panda=prediction.select('*').toPandas()

In [40]:
dataf=prediction.groupby('UserID','prediction').agg({'Rating':'mean'})
dataf_panda=dataf.select('*').toPandas()


In [41]:
pred_panda.head()


,UserID,MovieID,Rating,features,prediction
0,1,1193,5,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",12
1,1,661,3,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...",2
2,1,914,3,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",2
3,1,3408,4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",12
4,1,2355,5,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",16


In [42]:
dataf_panda.head()


,UserID,prediction,avg(Rating)
0,1,12,4.357143
1,9,24,3.000000
2,10,14,4.083333
3,20,7,3.666667
4,28,12,4.041667


In [43]:
df= pred_panda.merge(dataf_panda,how='left', left_on=['UserID','prediction'],right_on=['UserID','prediction'])


In [44]:
df.head()


,UserID,MovieID,Rating,features,prediction,avg(Rating)
0,1,1193,5,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",12,4.357143
1,1,661,3,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...",2,4.300000
2,1,914,3,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",2,4.300000
3,1,3408,4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",12,4.357143
4,1,2355,5,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",16,4.000000


In [45]:
df=df.rename(columns={'avg(Rating)':'avgRating'})


In [46]:
rmse=((df.Rating-df.avgRating)**2).mean()**0.5


In [47]:
rmse.real


0.9333231488364497